Character Sheet Reader

In [1]:
import sys
from pathlib import Path
import pandas as pd

# Add src/ to the Python path
sys.path.append(str(Path().resolve() / "src"))
sys.path.append(str(Path().resolve() / "data"))

In [4]:
from red_winter.modifier import Modifier

primary_effect = Modifier("Synthcoke", [("Archery",1)], ignore_list=["Synthcoke_addiction"])
secondary_effect = Modifier("Synthcoke_addiction", [("Archery",-2)])

composite_modifier = primary_effect + secondary_effect
composite_modifier.get_affect("Archery")


('+1', 'Synthcoke')

In [4]:
####################################################################

character_sheet_link = r"data/Solo_Crux.pdf" # change link to whatever pdf you're testing with

####################################################################

Organization/Reading of Text Fields in the PDF

In [5]:
from PyPDF2 import PdfReader


x = PdfReader(character_sheet_link)
x.get_form_text_fields()

{'Txt_Crit': None,
 'Txt_Addiction': '',
 'LVL DLV': '',
 'MOD DLV': '',
 'BASE DLV': '8',
 'LVL PAV': '',
 'MOD PAV': '',
 'BASE PAV': '8',
 'LVL PSV': '',
 'MOD PSV': '',
 'BASE PSV': '8',
 'LVL Riding': '',
 'MOD Riding': '',
 'BASE Riding': '8',
 'LVL Accounting': '',
 'MOD Accounting': None,
 'BASE Accounting': '4',
 'LVL AnimalHandling': '',
 'MOD AnimalHandling': '',
 'BASE AnimalHandling': '4',
 'MOD Bureaucracy': '',
 'LVL Business': '',
 'MOD Business': None,
 'BASE Business': '4',
 'LVL Composition': '',
 'BASE Composition': '4',
 'LVL Criminology': '',
 'MOD Criminology': None,
 'BASE Criminology': '4',
 'LVL Cryptography': '',
 'MOD Cryptography': None,
 'BASE Cryptography': '4',
 'LVL Deduction': '',
 'MOD Deduction': None,
 'BASE Deduction': '4',
 'LVL Education': '2',
 'MOD Education': None,
 'BASE Education': '6',
 'LVL Gamble': '',
 'MOD Gamble': None,
 'BASE Gamble': '4',
 'LVL Streetslang': '2',
 'MOD Streetslang': None,
 'BASE Streetslang': '6',
 'LVL Lang1': '4',


In [11]:
print(1 in range(2,9))

False


In [4]:
def determine_stat_group(skill_name: str, df: pd.DataFrame) -> str:
    bucket = "UNK"
    if skill_name in list(df["SkillName"]):
        bucket = df.loc[df["SkillName"] == skill_name, "AlignedStat"].iloc[0]
        bucket = bucket.strip()
    else:
        print(f"{skill_name} was not found :()")
    return bucket

In [5]:
#Mappings of Skillids to Canonical Names

maps = { "DLV" : "Drive Land Vehicle",
             "PAV" : "Pilot Air Vehicle",
             "PSV" : "Pilot Sea Vehicle",
             "RTD" : "Resist Torture/Drugs",
             "PhotoFilm" : "Photography/Film",
             "PDSculpt" : "Paint/Draw/Sculpt",
             "LVTech" : "Land Vehicle Tech",
             "SVTech" : "Sea Vehicle Tech",
             "AVTech" : "Air Vehicle Tech",
             "ESTech" : "Electronics/Security Tech",
             "Martial" : "Martial Arts",
             "Wilderness" : "Wilderness Survival",
             "Melee": "Melee Weapon",
             "Conceal" : "Conceal/Reveal Object",
             "Contortion" : "Contortionist",
             "WardrobeStyle" : "Wardrobe & Style",
             "ElecSec" : "Electronics/Security Tech",
             "WeaponsTech" : "Weaponstech"}

In [6]:
from PyPDF2 import PdfReader
from collections import defaultdict
import re

character_sheet_link = r'data/Solo_Crux.pdf'

character_sheet = PdfReader(character_sheet_link)
test_fields = character_sheet.get_form_text_fields()

player_defined_fields:list[str] = []

character_sheet_data = defaultdict(dict)
for category in ['Skills','Weapons','Gear','Cyberware','Totals']:
    character_sheet_data[category] = defaultdict(dict)
for category in ['Weapons','Gear','Cyberware']:
    character_sheet_data[category] = defaultdict(lambda: defaultdict(dict))

for field, value in test_fields.items(): # All 

    # Category 1: Top-Left Box, PG. 1
    if (field.startswith("Role") or
          field in ['Notes', 'Text Field176'] ):
        
        if field == 'Text Field176': # stores character name
            character_sheet_data["Handle"] = value

        character_sheet_data["Role"][field] = value

    # Category 2: STATs
    elif (field in ['INT','REF','DEX','TECH','COOL',
                  'WILL','LUCK-c','LUCK','MOVE','BODY',
                  'EMP MAX','EMP Curr', 'BODY-chargen']): 
        
        character_sheet_data["Stats"][field] = value

    # Category 3: Derived STATs
    elif (field in ['HPcurr', 'HPMax', 'SW_Value',
                    'DeathSave', 'HumanityCurr', 'HumanityMax']):

        character_sheet_data["Derived Stats"][field] = value


    # Category 2: All Skill LVL/MOD/BASEs
    elif (field.startswith("LVL") or 
          field.startswith("MOD") or
          field.startswith("BASE")):

        words:list[str] = field.split(" ")
        parameter, skill_name = words[0], " ".join(words[1:])


        '''what about AaaaaaaaaAaaaaa'''

        """skill_name = field_name.split()[-1]
                pattern = re.compile(r"[A-Z][a-z]*")
                variant_1 = re.findall(pattern, skill_name)
                skill_name = " ".join(variant_1)
"""
        # Alt 1: A skill has a known mapping that it should be renamed to.  
        if skill_name in maps.keys():
            skill_name = maps[skill_name]

        # Alt 2: A "custom" skill, filled by player.
        elif ('LocalExpert' in field or 'Science' in field or
                'PlayInstrument' in field or 'Lang' in field):
            
            variant_1 = "Txt_"+skill_name
            variant_2 = "Txt "+skill_name

            if variant_1 in test_fields:
                skill_name = test_fields[variant_1]
            elif variant_2 in test_fields:
                skill_name = test_fields[variant_2]
            else:
                skill_name = test_fields[skill_name]
   
        if not (skill_name == None or skill_name == ''): # Guards against empty custom skills
            character_sheet_data["Skills"][skill_name][parameter] = value
            
    # Category: Armor SP and Condition:
    elif ( field.startswith("SP") or 
           field.startswith("HPS")):
        
        if not field.endswith("Txt"): # Hard-coded, shouldn't load
            pass

    # Category 4: All fields related to Weapons on PG. 1
    elif field.startswith("WeaponTxt"):
        
        num:int = int(field[9:])
        index = num-1 if num < 35 else num

        header:str = ''
        
        match (index % 5):
            case 0:
                header = "weapon"      
            case 1:
                header = "DMG"  
            case 2:
                header = "Ammo"   
            case 3:
                header = "ROF" 
            case 4:
                header = "NOTES"

        character_sheet_data["Weapons"][index//5][header] = value

    # Category: IP
    elif field in ['IpMax','IpCurr']:

        character_sheet_data[field] = value

    # Category 5: Gear
    elif field.startswith("Gear"):
        
        text = ''.join(re.findall(r'\S', field))
        digits = int(''.join(re.findall(r'\d', field)))
        
        character_sheet_data["Gear"][digits][text[4:8]] = value

    # Category: Ammo
    elif field in ["Ammunition"]:
        character_sheet_data[field] = value

    # Category: Cash
    elif field in ["Cash", "cash20"]: # Cash seems unused?
        if field == "cash20":
            character_sheet_data["Cash"]["Amount"] = value
        else: #field == "Cash"
            character_sheet_data["Cash"]["Notes"] = value

    # Category 6: Cyberware & Additional Notes
    elif field.startswith("TextField") or field.startswith("Text Field"):
        digits = int(''.join(re.findall(r'\d', field)))

        thresholds = [5, 11, 17, 25, 33, 47, 59, 65, 71, 85, 99, 114, 127] 
        slots = ["Cyberaudio Suite", "Right Cybereye", "Left Cybereye",
                 "Right Cyberarm", "Left Cyberarm",
                 "Neuroport", "Neural Link", "Right Cyberleg", "Left Cyberleg",
                 "Internal", "External", "Fashionware", "Borgware"]
        
        for slot, threshold in zip(slots, thresholds):
            if digits <= threshold:

                column = "Name" if digits % 2 == 0 else "Data"

                # e.g. character_sheet/Cyberware/Neuroport/1/Name == "Biomonitor" 
                character_sheet_data["Cyberware"][slot][digits//2][column] = value
                break

        if digits > 127: # should be: 128-133; one of the big note pages PG. 4-6
            character_sheet_data["Additional Notes"][field] = value

    # Category 7: Lifepath
    elif ("Friends" in field or
          "TragicLove" in field or
          "Enemies" in field or field == "Ex-friendo1" or # both refer to enemies
          field in ["Cultural Origins", "Personality", "Clothing Style", "HairStyle",
                    "ValueMost", "FeelingAboutPeople", "ValuedPerson","ValuedPossession",
                    "FamilyBackground", "ChildhoodEnvironment", "Family Crisis", "Life Goals",
                    "RoleSpecificLifepath"]):
        
        if field == "Ex-friendo1":
            field = "Enemies1" # more consistent naming
        character_sheet_data["Lifepath"][field] = value
    
    # Category: HEAT
    elif field in ['Alias', 'RepEvents','Rep-Heat']:
        character_sheet_data["Heat"][field] = value

    # Category: Conditions
    elif field == 'Txt_Crit':
        character_sheet_data['Critical Injuries'] = value
    elif field == 'Txt_Addiction':
        character_sheet_data['Addictions'] = value

    # Category: Lifestyle
    elif (field in ['Housing', 'Rent', 'Lifestyle', 'Fashion']):
        character_sheet_data['Lifestyle'] = value

    # Category 8: Damage Taken
    elif (field in ['DMG Taken','BulletBrawl Head', 'MeleeMartial Body',
                    'MeleeMartial Head', 'BulletBrawl Body']):
        pass # Can't currently think of any meaningful reason to load this data

    # Category: Skill Totals
    elif field in ['TotalStats','skillsTotal','normSkills','doubleSkills']:
        character_sheet_data['Totals'][field] = value

    # Categories: Names of "custom" skills (redundant)
    elif ('LocalExpert' in field or 
          'Science' in field or
          'PlayInstrument' in field or
          'Lang' in field):
        
        player_defined_fields.append(field) # basically just don't print these as uncategorized

    # Uncategorized: print as debug
    else:
        print(f"{field} \t\t {value}")


    # SKILLs/STATS

    stat_associations = pd.read_csv(r"data/skills.csv")

    for skill_name in character_sheet_data['Skills'].keys():

        stat_group = determine_stat_group(skill_name, stat_associations)

        # Alt Path: The skill is player-defined (fill-in-blanks)
        if stat_group == "UNK": 
            for field in player_defined_fields:
                if test_fields[field] == skill_name:
                    canon_skill_name = field[4:-1]
                    stat_group = determine_stat_group(canon_skill_name, stat_associations)

        character_sheet_data['Skills'][skill_name]["STAT"] = stat_group

        
    
"""for category, members in character_sheet_data["Cyberware"].items():
    print(f"{category} --- {members}")"""

for skill, data in character_sheet_data["Skills"].items():
    print(f"{skill} \t {data["LVL"]}, {data["MOD"]}, {data["BASE"]}, {data["STAT"] }")


AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
AnimalHandling was not found :()
English was not found :()
AnimalHandling was not found :()
English was not f

In [7]:
#Prints the tree structure I've organized into

def print_dict_tree(d, indent="", is_last=True):
    """
    Recursively prints a dictionary as a tree structure.
    
    :param d: Dictionary or list to print
    :param indent: Current indentation string
    :param is_last: Boolean indicating if this is the last item in the current level
    """
    if isinstance(d, dict):
        items = list(d.items())
        for idx, (key, value) in enumerate(items):
            last = idx == len(items) - 1
            branch = "└── " if last else "├── "
            print(indent + branch + str(key))
            new_indent = indent + ("    " if last else "│   ")
            print_dict_tree(value, new_indent, last)
    elif isinstance(d, list):
        for idx, item in enumerate(d):
            last = idx == len(d) - 1
            branch = "└── " if last else "├── "
            print(indent + branch + f"[{idx}]")
            new_indent = indent + ("    " if last else "│   ")
            print_dict_tree(item, new_indent, last)
    else:
        # Leaf node
        print(indent + "└── " + str(d))


print_dict_tree(character_sheet_data)


├── Skills
│   ├── Drive Land Vehicle
│   │   ├── LVL
│   │   │   └── 
│   │   ├── STAT
│   │   │   └── REF
│   │   ├── MOD
│   │   │   └── 
│   │   └── BASE
│   │       └── 8
│   ├── Pilot Air Vehicle
│   │   ├── LVL
│   │   │   └── 
│   │   ├── STAT
│   │   │   └── REF
│   │   ├── MOD
│   │   │   └── 
│   │   └── BASE
│   │       └── 8
│   ├── Pilot Sea Vehicle
│   │   ├── LVL
│   │   │   └── 
│   │   ├── STAT
│   │   │   └── REF
│   │   ├── MOD
│   │   │   └── 
│   │   └── BASE
│   │       └── 8
│   ├── Riding
│   │   ├── LVL
│   │   │   └── 
│   │   ├── STAT
│   │   │   └── REF
│   │   ├── MOD
│   │   │   └── 
│   │   └── BASE
│   │       └── 8
│   ├── Accounting
│   │   ├── LVL
│   │   │   └── 
│   │   ├── STAT
│   │   │   └── INT
│   │   ├── MOD
│   │   │   └── None
│   │   └── BASE
│   │       └── 6
│   ├── AnimalHandling
│   │   ├── LVL
│   │   │   └── 
│   │   ├── STAT
│   │   │   └── UNK
│   │   ├── MOD
│   │   │   └── 
│   │   └── BASE
│   │       └── 6
│   ├── Bureaucracy
│